In [2]:
import csv
import xml.etree.ElementTree

import re


In [7]:
filename_coord = 'gnaf/australian_postcodes.csv'
location = dict()

with open(filename_coord, 'rb') as in_f:
    loc = csv.reader(in_f, delimiter=',', quotechar='"')
    
    for i, row in enumerate(loc):
        if i == 0: continue
            
        #print row
        postcode, sub, state, lon, lat, _, _ = row
        location[sub+'_'+state] = [lat, lon]
        


In [10]:
for k in location.keys():
    print k, location[k]
    break
    


COOMRITH_QLD ['-27.383801', '149.525379']


In [3]:
filename = 'act_population/ACT_Population_Projections_by_Suburb__2015_-_2020_.csv'


In [59]:
# age 0-4 row[2:7], row[88:93]
# age 5-17 row[2:7], row[88:93]
# age 18-60 row[2:7], row[88:93]
# age 61+ row[2:7], row[88:93]

act_pop_year = dict()

with open(filename) as in_f:
        act_p = csv.reader(in_f, delimiter=',', quotechar='"')
        
        for i, row in enumerate( act_p ):
            
            year = (row[0]).split()[0][6:]
            
            #print row[63:88], row[149:174] 
            suburb = row[1].upper().strip()
            
            
            if i >= 1:
                age_0_4 = sum( [int(r) for r in row[2:7]] + [int(r) for r in row[88:93]] )
                age_5_17 = sum( [int(r) for r in row[7:20]] + [int(r) for r in row[93:106]] )
                age_18_60 = sum( [int(r) for r in row[20:63]] + [int(r) for r in row[106:149]] )
                age_61_plus = sum( [int(r) for r in row[63:88]] + [int(r) for r in row[149:174]] )
                
                lat = lon = ''
                if suburb+'_ACT' in location: 
                    lat, lon = location[suburb+'_ACT']
                    
                #print age_0_4, age_5_17, age_18_60, age_61_plus, suburb, lat, lon
                act_pop_year[suburb+'_'+year] = (age_0_4, age_5_17, age_18_60, age_61_plus, lat, lon)
            
            #print row, year, '\n'
            
            #if i >= 2: break
print(len(act_pop_year.keys()))
        

660


In [85]:
# Grant data
filename_grants = 'grants_pbi_mk1.csv'
filename_grants_out = 'grants_pbi_mk2_temp.csv'

header = ['\xef\xbb\xbfAgency', 'Column2', 'GA ID', 'Column4', 'Column5', 'Column6', 'Grant received', 
          'Column8', 'Column9', 'Funds received', 'Column11', 'Column12', 'Column13', 'Outcome', 
          'Grant Name', 'Description', 'Column17', 'Column18', 'Column19', 'Column20', 'Type', 'Category', 
          'Column23', 'Column24', 'Column25', 'Column26', 'Column27', 'Column28', 'Business Name', 'ABN', 
          'Suburb', 'Column32', 'PostCode', 'State', 'Column35', 'Column36', 'Column37', 'Column38', 
          'Column39', 'Column40', 'Column41', 'Latitude', 'Longitude', 'Population_0_4',
          'Population_5_17', 'Population_18_60', 'Population_61_plus', 'Population_lat', 'Population_lon']

with open(filename_grants) as in_f:
    with open(filename_grants_out, 'w') as out_f:
        grants = csv.reader(in_f, delimiter=',', quotechar='"')
        out_f.write('\t'.join( header ) + '\n')
        
        for i, row in enumerate( grants ):
            suburb = row[30].upper().strip()
            state = row[33].upper().strip()
            year = row[6][-4:]
            ABN = row[29].replace(' ', '')
            row[29] = ABN
            
            #print suburb, state, year, ABN
            # act_pop_year[suburb+'_'+year] = (age_0_4, age_5_17, age_18_60, age_61_plus, lat, lon)
            age_0_4 = ''
            age_5_17 = ''
            age_18_60 = ''
            age_61_plus = ''
            lat = ''
            lon = ''
            
            if state == 'ACT' and suburb+'_'+year in act_pop_year:
                age_0_4, age_5_17, age_18_60, age_61_plus, lat, lon = [str(n) for n in act_pop_year[suburb+'_'+year] ]
                #print age_0_4, age_5_17, age_18_60, age_61_plus, lat, lon
                #print row
            
            if i == 0: continue #header = row
                
            out_f.write('\t'.join( row + [age_0_4, age_5_17, age_18_60, age_61_plus, lat, lon] ) + '\n')
            
                
            #print row, 
            #if i > 10: break
        